In [ ]:
import torch
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from matplotlib import pyplot as plt

In [ ]:
train_data = datasets.FashionMNIST(
    root="../fasionMNIST/data",
    train=True,
    download=False,
    transform=ToTensor())

test_data = datasets.FashionMNIST(
    root="../fasionMNIST/data",
    train=False,
    download=False, 
    transform=ToTensor()
)

batch_size = 64
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)


In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 5), # 28 - 5 + 1 = 24
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 24 / 2 = 12
            nn.Conv2d(6, 16, 5), # 12 - 5 + 1 = 8
            nn.ReLU(),
            nn.MaxPool2d(2, 2) # 8 / 2 = 4
        )
        self.fc = nn.Sequential(
            nn.Linear(16*4*4, 120),  # 十六通道特征图，每个都是4 × 4大小
            nn.ReLU(),
            nn.Linear(120, 84), 
            nn.ReLU(),
            nn.Linear(84, 10),  
        )
    def forward(self, X):
        feature = self.conv(X)
        output = self.fc(feature.view(X.shape[0], -1))
        return output


In [ ]:
model = LeNet()
print(model)

In [ ]:
for name, param in model.conv.named_parameters():
    print(name, param.shape)
print()
for name, param in model.fc.named_parameters():
    print(name, param.shape)

In [ ]:
loss_fn = nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
loss_list = []

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:  
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:.6f}  [{current:5d}/{size:5d}]")

def test(dataloader, model, loss_fn):
    model.eval()
    loss, correct = 0, 0
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).float().sum().item()
        loss /= num_batches
        loss_list.append(loss)
        correct /= size
    print("Avg Loss: {:.6f}\tAccuracy: {:.2f}%".format(loss, correct*100))        
    

In [ ]:
epoches = 50
for epoch in range(epoches):
    print("Epoch {}\n-------------".format(epoch+1))
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)



In [ ]:
def plotLoss(epoches, loss_list):
    plt.scatter(range(epoches), loss_list, c='r', s=1, label='Lost of epoch')
    plt.legend(loc='upper right')
    plt.title('Loss with training')
    plt.show()

plotLoss(epoches, loss_list)